<a href="https://colab.research.google.com/github/MFaiqKhan/MNIST_modelclassif/blob/main/MNIST_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import torch
import torchvision # package in pytoch providing datasets, transforms and models for working with computervision tasks

In [47]:
print("torch", torch.__version__)

torch 1.13.1+cu116


In [48]:
# loading the datasets
from torchvision.datasets import MNIST # contains popular datasets for cv tasks, from there we are importing MNIST dataset


In [49]:
# downloading MNIST train and test dataset into local aswell loaded into the variables

data_train = MNIST(root="./", download=True, train=True)
data_test = MNIST(root="./", download=True, train=False) # to download the test, simply set train to false

In [50]:
data_train[10][0] # second index, zero is the image

In [51]:
data_train[10][1] # second index, 1 represents the actual number

3

In [52]:
type(data_train[10][0]) # PIL is a class in python Imaging Library (PIL) 
# powerful tool for working with image data in Python, and is commonly used in a variety of 
# applications such as image processing, computer vision, and machine learning.

PIL.Image.Image

In [53]:
print(data_train[0][0].size) # each image is a handwritten digit that has been resized to a 28x28 pixel grayscale image
# represents the width and height of the image in pixels.


(28, 28)


In [54]:
# building a dense neural network with 2 layers, not any CNN, 
# so the data should be in vector form not whole 28 by 28 pixel can inserted.

A dense neural network, also known as a fully connected neural network, is a type of artificial neural network where each neuron in one layer is connected to every neuron in the next layer. In a dense neural network, the input layer receives input data, which is then passed through one or more hidden layers before reaching the output layer. The output layer generates the final output of the model.

In a dense neural network, the connections between neurons are represented by weights, which are learned during training using an optimization algorithm such as backpropagation. During training, the network adjusts the weights to minimize a loss function, which measures the difference between the predicted output and the true output.

The number of neurons in each layer, as well as the number of layers, are hyperparameters that can be tuned to improve the performance of the model. A deeper network with more hidden layers can learn more complex representations of the data, but may also be more difficult to train and prone to overfitting.

Dense neural networks are widely used in a variety of applications, such as image classification, natural language processing, and speech recognition. They are popular because of their flexibility and ability to learn complex patterns in the data. However, they may not be suitable for all types of data, and other types of neural networks, such as convolutional neural networks (CNNs) or recurrent neural networks (RNNs), may be more appropriate for certain tasks

In [55]:
# normlizing the dataset

# Contains a set of functions for image transformations, such as resizing, cropping, and flipping.
from torchvision.transforms import Lambda , ToTensor, Compose 

# Apply this transformation on the top of dataset
Transform = Compose([
    ToTensor(), # converting PIL image to tensor
    Lambda(lambda image: image/255), # converting every pixel of the image to range of 0 and 1, normalizing it basically
    Lambda(lambda image: image.view(784))
])

# lets say we are downloading the dataset for the firsttime , we can actually apply the transform function directly while downloading so
# we will get transformed values

data_train = MNIST(root="./", download=True, train=True, transform = Transform)
data_test = MNIST(root="./", download=True, train=False, transform = Transform)


This code is part of a Python script that is preparing the MNIST dataset for machine learning model training. Specifically, it is normalizing the pixel values of the images in the dataset by dividing them by 255, which is the maximum pixel value.

The code is using the PyTorch torchvision module, which contains a set of functions for image transformations. It first imports three of these functions: Lambda, ToTensor, and Compose.

ToTensor is a transformation function that converts a PIL image or NumPy array to a PyTorch tensor. This is necessary because PyTorch models require tensor inputs.

Lambda is a transformation function that applies a user-defined lambda function to the input. In this case, it is used to divide each pixel value by 255, which normalizes the pixel values to the range of [0, 1].

Compose is a transformation function that combines multiple transformations into a single transform. In this case, it combines the ToTensor and Lambda transformations.

The Transform object created using Compose is then passed as an argument to the transform parameter of the MNIST dataset object. This applies the transform to each image in the dataset as it is loaded, so the model will receive normalized input data.

The data_train and data_test objects created using MNIST contain the training and test datasets, respectively. They are downloaded from the specified root directory if they don't already exist on the system.

Overall, this code is a standard preprocessing step for working with image data in PyTorch, and it ensures that the input data is normalized and in the correct format for use in a machine learning model.
-------------------------------------------------------------

In the provided code, image.view(784) is being used to reshape the input image tensor into a one-dimensional tensor of size 784. This is because the original image tensor has dimensions of (1, 28, 28), where the first dimension represents the number of channels (1 for grayscale), and the second and third dimensions represent the height and width of the image.

By using view(784), the image tensor is flattened into a vector of length 784, which represents a single feature vector for the input image. The number 784 comes from 28 x 28, which is the total number of pixels in each image in the MNIST dataset.

This step is necessary because most machine learning models require input data to be in a flattened, one-dimensional format. By reshaping the input image tensor in this way, we can pass it directly into a machine learning model for training or prediction.

Flattening the image is necessary because most machine learning models require input data to be in a flattened, one-dimensional format. In the case of image data, this means that the image must be reshaped into a vector or array of pixel values.

When working with image data, we often represent each image as a matrix or tensor, where each element of the matrix represents a pixel in the image. However, many machine learning algorithms require input data to be in a one-dimensional format, where each feature is represented by a single value.

Flattening the image allows us to represent each image as a single feature vector, where each element of the vector represents a pixel value in the image. This makes it easier to pass the image data into a machine learning model, as the model expects a one-dimensional array of features as input.

In the case of the MNIST dataset, each image is a 28 x 28 grayscale image, which means it contains 784 pixels. By flattening the image into a one-dimensional vector of size 784, we can represent each image as a single feature vector of length 784. This allows us to pass the image data directly into a machine learning model for training or prediction.

In [56]:
data_train[0][0].shape

torch.Size([784])

In [57]:
# Building the model through nn.module 

from torch import nn, optim 

class Model(nn.Module): #defining new class model , inheriting from nn.Module Class, which is base call for all neural networks modules in PyTorch.
  def __init__(self, sizes): #taking layer sizes # [128, 64, 10]
    super().__init__()
    self.hidden_layer_1 = nn.Linear(784, sizes[0])# (input channels, output channels) , # [784, 128]
    self.act1 = nn.Sigmoid()
    self.hidden_layer_2 = nn.Linear(sizes[0], sizes[1]) # 128, 64
    self.act2 = nn.Sigmoid()
    self.output_layer = nn.Linear(sizes[1], sizes[2]) # 128, 10
    self.act3 = nn.Softmax(dim=1)

    self.loss = nn.CrossEntropyLoss() # calculating loss
    self.optimizer = optim.Adam(self.parameters()) # weights are the learnable params

  def forward(self, inputs):
      x = self.hidden_layer_1(inputs)
      x = self.act1(x)
      x = self.hidden_layer_2(x)
      x = self.act2(x)
      x = self.output_layer(x)
      x = self.act3(x)
      return x

  def fit(self, X,Y):
      self.optimizer.zero_grad()
      y_pred = self.forward(X)
      loss = self.loss(y_pred, Y)
      loss.backward() # calculation of gradients - backpropagation
      self.optimizer.step() # updating the weights
      return loss.item()

  # Predict and evalute for testing purposes
  def predict(self, X):
      with torch.no_grad():
        return torch.argmax(self.forward(X), axis=1)
      
  def evaluate(self, test_dataloader):
      correct = 0
      for X,Y in test_dataloader:
        y_pred = self.predict(X)
        correct += (Y == y_pred).sum() #batch
      acc = correct / (len(test_dataloader)* 16)
      print(f"Accuracy: {acc}")



class Model(nn.Module): - This line defines a new class Model that inherits from the nn.Module class, which is the base class for all neural network modules in PyTorch.

def __init__(self, sizes): - This is the constructor method for the Model class, which takes a single argument sizes. sizes is a list containing three integers, representing the sizes of the input layer, hidden layer, and output layer, respectively.

super().__init__() - This line calls the constructor of the parent class (nn.Module) to initialize the instance of the Model class.

self.hidden_layer_1 = nn.Linear(784, sizes[0]) - This line creates the first hidden layer of the neural network using the nn.Linear class. The nn.Linear class takes two arguments: the size of the input layer (784, which corresponds to the flattened size of the MNIST images), and the size of the output layer (sizes[0], which is the size of the first hidden layer).

self.act1 = nn.Sigmoid() - This line defines the activation function for the first hidden layer. In this case, we are using the sigmoid activation function, which maps any input value to a value between 0 and 1.

self.hidden_layer_2 = nn.Linear(sizes[0], sizes[1]) - This line creates the second hidden layer of the neural network, which takes the output of the first hidden layer as input.

self.act2 = nn.Sigmoid() - This line defines the activation function for the second hidden layer, which is also the sigmoid activation function.

self.output_layer = nn.Linear(sizes[1], sizes[2]) - This line creates the output layer of the neural network, which takes the output of the second hidden layer as input.

self.act3 = nn.Softmax(dim=1) - This line defines the activation function for the output layer, which is the softmax activation function. The dim=1 argument specifies that the softmax function should be applied to the second dimension of the output tensor, which corresponds to the different classes.

self.loss = nn.CrossEntropyLoss() - This line defines the loss function for the neural network. In this case, we are using the cross-entropy loss function, which is commonly used for multi-class classification problems.

self.optimizer = optim.Adam(self.parameters()) - This line defines the optimizer used to update the weights of the neural network during training. In this case, we are using the Adam optimizer, which is an adaptive learning rate optimization algorithm

n Python, self is a reference to the current instance of the class. In this case, self refers to the instance of the Model class that is being created by the constructor method __init__(). By using self, we can access and modify the instance variables and methods of the class within the constructor and other methods of the class. For example, self.hidden_layer_1 refers to the Linear layer that is created as an instance variable within the __init__() method, and can be accessed and used in other methods of the class such as the forward pass method.

The sigmoid function is a commonly used non-linear activation function in neural networks. It takes any real-valued input and outputs a value between 0 and 1, which can be interpreted as a probability.

The softmax function is a commonly used activation function in the output layer of a neural network for multiclass classification problems. It takes a vector of real-valued scores as input and outputs a probability distribution over the classes.

In the line self.act3 = nn.Softmax(dim=1), we are defining the activation function for the output layer of our neural network model. The softmax activation function is used when we have a multi-class classification problem and want to obtain class probabilities as the output.

The dim argument in the nn.Softmax(dim=1) specifies the dimension along which the softmax function should be applied. In PyTorch, tensors have multiple dimensions, and we need to specify the dimension along which we want to apply the softmax function. In the case of our output tensor, the first dimension represents the number of samples in the batch, while the second dimension represents the number of output classes. Therefore, we want to apply the softmax function along the second dimension (i.e., dim=1).

In the context of a neural network, the output classes refer to the categories or classes that we want the network to predict. For example, in the case of the MNIST dataset, we have 10 output classes corresponding to the digits from 0 to 9. Therefore, the output of the neural network for each image in the dataset would be a probability distribution over the 10 output classes, indicating the likelihood of the image belonging to each class.

In our code, the number of output classes is determined by the size of the output layer of the neural network. In the Model class, the size of the output layer is specified as sizes[2], which is the third element in the sizes list passed to the constructor. Therefore, the number of output classes is determined by the value of sizes[2].

Class probabilities refer to the probability of each possible class label given a set of input features. In other words, for a multi-class classification problem with k possible classes, the softmax function is used to compute a probability distribution over the k classes for a given input.

For example, in the case of the MNIST dataset, we have 10 possible classes (digits 0 to 9). When we pass an image of a digit to our neural network, we want it to output a probability distribution over these 10 classes indicating the likelihood of the image belonging to each class. The softmax function is used to convert the output of the neural network (which is in the form of raw scores or logits) into a probability distribution over the 10 classes. Each element in the output vector represents the probability of the corresponding class given the input image. The probabilities for all the classes sum up to 1.0.

In machine learning, the goal is to create a model that can predict the output for a given input. During the training process, the model is adjusted to minimize the difference between its predicted output and the actual output (i.e., the ground truth).

The loss function is used to measure the difference between the predicted output and the actual output. The goal of the optimization algorithm is to minimize this loss function.

For multi-class classification problems, where there are multiple possible outcomes, cross-entropy loss is a commonly used loss function. It measures the dissimilarity between the predicted probability distribution and the actual probability distribution of the output.

In the case of the MNIST classification problem, there are 10 possible output classes (digits 0-9). The cross-entropy loss function calculates the difference between the predicted probability distribution (i.e., the output of the softmax function) and the actual probability distribution (i.e., the ground truth). It penalizes the model more heavily when it predicts the wrong class with high confidence and less when it predicts the wrong class with low confidence.

In PyTorch, the nn.CrossEntropyLoss() function combines the softmax activation function and the cross-entropy loss function into a single operation. It takes the logits (i.e., the output of the last layer of the neural network before the softmax activation) and the ground truth labels as inputs and returns the cross-entropy loss between the predicted probability distribution and the actual probability distribution. The optimization algorithm uses the gradients of the loss function with respect to the model parameters to update the weights and biases of the model during training.

he optim.Adam function is part of the PyTorch library and creates an optimizer object that can be used to update the weights of the neural network during training. The self.parameters() method returns a list of all the learnable parameters of the model, which are then passed to the optimizer. When the optimizer's step() method is called, it updates the values of the model's parameters based on the gradients computed during the backpropagation step.

__forward pass method__ : This is the forward method of the Model class, which is responsible for performing the forward pass of the neural network. The forward pass is the process of taking the input data and propagating it through the network to obtain the output.

In this method, the input data is passed through the first hidden layer (self.hidden_layer_1) using the linear transformation defined by the weights and biases of this layer. Then, the output of this layer is passed through the sigmoid activation function (self.act1) to introduce non-linearity in the network.

The output of the first hidden layer is then passed through the second hidden layer (self.hidden_layer_2) using another linear transformation, followed by another sigmoid activation function.

Finally, the output of the second hidden layer is passed through the output layer (self.output_layer), which is another linear transformation, and the output of this layer is passed through the softmax activation function (self.act3) to obtain the class probabilities for each input sample.

The output of the softmax activation function is then returned by the forward method.

__fit method__ : This code block defines the fit method for the Model class, which is used to train the neural network on the input data X with corresponding labels Y.

Here's a step-by-step breakdown of the code:

self.optimizer.zero_grad(): This line clears the gradients of all optimized tensors (i.e., all learnable parameters) to prepare for a new optimization step. This is necessary because PyTorch accumulates gradients by default, so we need to explicitly set them to zero before calculating new gradients for a new batch of data.

y_pred = self.forward(X): This line computes the predicted output y_pred for the input data X by passing it through the neural network. The forward method defined earlier in the Model class is used for this purpose.

loss = self.loss(y_pred, Y): This line computes the loss between the predicted output y_pred and the actual labels Y. The self.loss attribute of the Model class is an instance of the CrossEntropyLoss class, which is commonly used for multi-class classification problems.

loss.backward(): This line performs backpropagation to calculate the gradients of the loss with respect to all learnable parameters in the neural network. These gradients will be used to update the weights of the neural network in the next step.

self.optimizer.step(): This line updates the weights of the neural network based on the gradients computed in the previous step. The self.optimizer attribute of the Model class is an instance of the Adam optimizer, which is a popular optimization algorithm used for training neural networks.

return loss.item(): This line returns the scalar value of the loss as a Python float. The item() method is used to extract the scalar value from the PyTorch tensor object. This value is used to track the training progress of the neural network over time.

__Predict and evalute__:  The above code defines two methods, predict() and evaluate(), to make predictions and evaluate the performance of the neural network on the test dataset.

The predict() method takes the input tensor X, which represents the test dataset, and makes predictions using the forward() method of the neural network. It returns the indices of the class with the highest probability using the argmax() function along the second dimension.

The evaluate() method takes the test dataset as input, which is passed through the test_dataloader object. The method initializes a variable named correct to 0, which will keep track of the number of correct predictions. It then iterates through the test dataset using a for loop and makes predictions using the predict() method. The predictions are compared against the ground truth labels Y using the == operator, and the number of correct predictions is accumulated in the correct variable. Finally, the accuracy is calculated by dividing the number of correct predictions by the total number of samples in the test dataset, which is len(test_dataloader)*16 (batch size). The accuracy is printed to the console using the print() function.

The code above contains two functions predict and evaluate which are used to evaluate the performance of the neural network model on test data.

The predict function takes a batch of input data X and returns the predicted class labels for each input. The function first passes the input data X through the forward method of the model to obtain the output probabilities for each class. The argmax method is then used to obtain the index of the highest probability value, which corresponds to the predicted class label. The no_grad context manager is used to ensure that no gradients are computed during the prediction step, which helps to save memory and speed up the computation.

The evaluate function takes a dataloader of test data and calculates the accuracy of the model on this data. The function iterates over each batch of test data and uses the predict function to obtain the predicted class labels for each input in the batch. The number of correctly predicted samples is then counted and the accuracy is computed as the ratio of correctly predicted samples to the total number of samples in the test set. Finally, the accuracy value is printed to the console.

In [58]:
#Initiliazing the model:

model = Model([128,64,10])

In [59]:
from torch.utils.data import DataLoader

BATCH = 16
train_dataloader = DataLoader(data_train, batch_size=BATCH, shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=BATCH, shuffle=True)

In machine learning, data is often split into training and testing sets. The training set is used to train the model, while the testing set is used to evaluate the performance of the model on unseen data.

torch.utils.data.DataLoader is a utility class that allows us to load data in batches for training and testing. It takes a dataset as input and returns an iterator that can be used to iterate over the data in batches.

In the above code snippet, we are creating two DataLoader objects: train_dataloader and test_dataloader. data_train and data_test are PyTorch datasets containing the training and testing data, respectively.

The batch_size parameter specifies the number of samples in each batch. A smaller batch size will result in more updates to the model weights, but each update will be less accurate. A larger batch size will result in fewer updates to the model weights, but each update will be more accurate.

The shuffle parameter specifies whether to shuffle the data after each epoch. Shuffling the data can help prevent overfitting and improve the generalization of the model.

In summary, the DataLoader objects created in the code snippet allow us to load the training and testing data in batches, which is useful for training and evaluating our neural network model.

Overfitting occurs when a machine learning model is trained too well on the training dataset, to the extent that it starts to memorize and adapt to the specific features and noise of the training data rather than learning the underlying patterns and generalizing to new data. This results in poor performance on unseen data, as the model has not learned to identify the general patterns and relationships in the data but instead has simply memorized the training data.

Overfitting can be caused by several factors such as using a model that is too complex, having too few training examples, or training for too many epochs. It is important to identify and mitigate overfitting to ensure that the model performs well on unseen data.

In machine learning, an epoch refers to a complete pass through a given dataset during the training phase of a model. During an epoch, the model is exposed to every training sample once and updates its parameters based on the loss function. One epoch consists of one forward pass and one backward pass (i.e., one iteration) for all training samples. The number of epochs is a hyperparameter that needs to be chosen in order to train a model effectively. Typically, the number of epochs is chosen based on the training data size, the complexity of the model, and the computational resources available.

In [60]:
# Training the Model:

from tqdm import tqdm
epochs = 10
for i in range(epochs):
  total_loss = 0
  for X,Y in tqdm(train_dataloader, desc = f"Fitting epoch {i}"):
    loss = model.fit(X,Y)
    total_loss += loss
  avg_loss = total_loss/len(train_dataloader)
  print(f"Epoch {i}: {avg_loss:.4f}")

Fitting epoch 0: 100%|██████████| 3750/3750 [00:29<00:00, 128.76it/s]


Epoch 0: 2.1896


Fitting epoch 1: 100%|██████████| 3750/3750 [00:28<00:00, 132.11it/s]


Epoch 1: 1.9470


Fitting epoch 2: 100%|██████████| 3750/3750 [00:29<00:00, 125.72it/s]


...

Epoch 7: 1.6354


Fitting epoch 8: 100%|██████████| 3750/3750 [00:29<00:00, 128.96it/s]


Epoch 8: 1.6247


Fitting epoch 9: 100%|██████████| 3750/3750 [00:28<00:00, 131.13it/s]

Epoch 9: 1.6168


This code trains the model using a loop that iterates over multiple epochs (the number of times the entire dataset is passed forward and backward through the model).

The outer loop goes from 0 to 9 (10 epochs) and the inner loop iterates over the train_dataloader object, which contains the training data divided into batches of size 16.

For each batch, the fit() method of the model object is called with the input tensor X and the target tensor Y. The fit() method calculates the loss and updates the weights of the model.

The total_loss variable is used to keep track of the total loss for the current epoch. The avg_loss variable is calculated by dividing the total loss by the number of batches in the train_dataloader.

Finally, the average loss for the epoch is printed to the console using a formatted string. The desc parameter of the tqdm function is used to display a progress bar with a message indicating the current epoch being trained.

tqdm is a Python library that provides a progress bar for loops. It is used to visualize the progress of the loop and estimate the remaining time for completion. It can be used with any iterable in Python, such as lists, tuples, and generators. tqdm stands for "progress" in Arabic, and it was created by Noam Yorav-Raphael in 2016.

In [61]:
model.evaluate(test_dataloader)

Accuracy: 0.8565999865531921


In [62]:
# saving the model(only weights)

path = "./mnist_model.pth"
torch.save(model.state_dict(), path) # Only Weights

In [63]:
# To load the model weights , we first have to initialize the model with same config 

model_2 = Model([128,64,10])
model_2.load_state_dict(torch.load(path))

<All keys matched successfully>

In [64]:
# Now If I evalute the model, it will give me same accuracy as the original model

model_2.evaluate(test_dataloader)

Accuracy: 0.8565999865531921


In [65]:
# I want to save the whole model and not just weights

path = "./full_model.pth"
torch.save(model, path) # full model with whole architecture(source code and weights)


In [66]:
model_3 = torch.load(path)

In [67]:
model_3.evaluate(test_dataloader)

Accuracy: 0.8565999865531921


In [68]:
# Visualizing the evaluate function in the model:

for X,Y in test_dataloader:
  y_pred = model.predict(X)
  print(Y, y_pred)
  print(Y==y_pred)
  break

  # the evaluate function gives us the accuracy by summing up the true/false table, and total true available

  # here we can see that what value matches and what doesnt matches through false and true

tensor([4, 6, 3, 3, 3, 3, 8, 9, 3, 6, 6, 8, 0, 7, 4, 9]) tensor([4, 6, 3, 7, 3, 3, 8, 9, 3, 6, 2, 8, 0, 7, 4, 9])
tensor([ True,  True,  True, False,  True,  True,  True,  True,  True,  True,
        False,  True,  True,  True,  True,  True])
